In [1]:
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm

In [2]:
pos = []
with open("poststem2.txt") as f:
    lines = f.readlines()
    for line in lines:
        line = line.split(',')[0]
        pos.append(line)

In [3]:
kata_kata = {kata:1 for kata in pos}

In [4]:
with open('budi/negstem2.txt') as f:
    lines = f.readlines()
    for line in lines:
        line = line.split(',')[0]
        try:
            if kata_kata[line] != 1:
                kata_kata[line] = -1
        except KeyError:
            kata_kata[line] = -1

In [5]:
df = pd.read_csv('budi/stemmed_kpu_no_jutsu.csv')

In [6]:
del df['Unnamed: 0']

In [7]:
words = df['Stemmed']

In [8]:
def stem(ab):
    a = ab.split()
    a = [re.sub(r'(mem|kan|ter|per)', '', b) for b in a]
    return ' '.join(a)

In [9]:
for i, ws in enumerate(words):
    words.iloc[i] = stem(ws)

In [10]:
scores = []
for ws in tqdm(words):
    wss = ws.split()
    sc = 0
    i = 0
    for w in wss:
        try:
            sc += kata_kata[w]
            i += 1
        except:
            continue
    if i != 0:
        scores.append(sc/i)
    else:
        scores.append(0.0)

In [12]:
i = 0
for s in scores:
    if s == 0:
        i += 1

In [29]:
mask = scores != 0.0

In [30]:
words = words[mask]

In [31]:
scores = scores[mask]

In [12]:
import numpy as np

In [13]:
scores = np.array(scores)

In [14]:
for i, s in enumerate(scores):
    if s > 0:
        scores[i] = 1
    elif s < 0:
        scores[i] = -1
    else:
        scores[i] = 0

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split

In [36]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=3.0)
model = GridSearchCV(SVC(), {'gamma':[1, 0.1, 0.001, 0.0001, 0.00001],
                                         'kernel':['linear', 'rbf']},
                                 refit=True, verbose=0)

In [37]:
words_f = vectorizer.fit_transform(raw_documents=words)

In [38]:
x_train, x_test, y_train, y_test = train_test_split(words_f, scores,
                                                            test_size=0.2, shuffle=True)

In [39]:
model.fit(x_train, y_train)

c:\users\tuan tamvan\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'gamma': [1, 0.1, 0.001, 0.0001, 1e-05],
                         'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [40]:
model = model.best_estimator_

In [41]:
model.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [42]:
y_pred = model.predict(x_test)

In [43]:
report = classification_report(y_test, y_pred, output_dict=True)

In [44]:
report

{'-1.0': {'precision': 0.8024691358024691,
  'recall': 0.8666666666666667,
  'f1-score': 0.8333333333333334,
  'support': 300},
 '1.0': {'precision': 0.7969543147208121,
  'recall': 0.7104072398190046,
  'f1-score': 0.7511961722488039,
  'support': 221},
 'accuracy': 0.800383877159309,
 'macro avg': {'precision': 0.7997117252616406,
  'recall': 0.7885369532428357,
  'f1-score': 0.7922647527910687,
  'support': 521},
 'weighted avg': {'precision': 0.8001298354971982,
  'recall': 0.800383877159309,
  'f1-score': 0.7984920423550589,
  'support': 521}}

In [15]:
scores

array([ 0.,  0.,  1., ...,  1., -1., -1.])

In [27]:
pd.DataFrame({
    "Tweet" : words,
    "Scores" : scores
}).to_csv("kpu_scored1_fix.csv")

In [20]:
df1 = pd.DataFrame({
    "Tweet" : words,
    "Scores" : scores
})

In [21]:
df1 = df1[df1["Scores"] != 0.0]

In [22]:
df1

,Tweet,Scores
2,kpuid bawasluri mantap,1
3,legowo sama hasil situng emng kalah legowo pil...,1
4,liat timeline kpu bakal tabrak hehe,-1
5,komplek mega mai mana suaranya salah input kpu...,-1
6,tps gerung utara gerung lombok barat ntb input...,-1
7,malah lebih dahsyat salah inputnya,-1
8,salah kpu input data formulir c manusiawi,-1
9,kpuid sedikit bocor,-1
10,salah kpuid kpu tanya komisi ii dpr pilih manu...,-1
11,rakyat surabaya tuntut suara dirampok inilah h...,-1


In [23]:
df1.to_csv("kpu_scored_no_netral_coba.csv")

In [30]:
dfpos = pd.read_csv("kpu_scored_no_netral_fix.csv")

In [31]:
dfpos.rename({"Unnamed: 0":"a"}, axis="columns",inplace=True)

In [32]:
del dfpos["a"]

In [33]:
dfpos = dfpos[dfpos["Scores"] != -1.0]

In [34]:
dfpos.to_csv("kpu_postif_fix.csv")

In [35]:
dfpos



,Tweet,Scores
0,kpuid bawasluri mantap,1.0
1,legowo sama hasil situng emng kalah legowo pil...,1.0
12,kpkri bap udah revisi situng kpu direvisi bene...,1.0
17,dukung cinta indonesia adil makmur curang pemi...,1.0
22,bilang kalian lembaga survey bayar atur quick ...,1.0
23,nyata duka citasimpati atas jadi jokowi jancuk,1.0
25,guru besar ui kpu layak jadi hasil pemilu,1.0
27,lama nongol romy sampai selamat kpu laksana pe...,1.0
28,menang telak jawa barat jawa timur dki banten ...,1.0
30,insya allah senin besok mulai puasa ramadhan s...,1.0


In [26]:
dfsvm = pd.read_csv("kpu_scored_no_netral_fix.csv")

In [100]:
dfsvm.rename({"Unnamed: 0":"a"}, axis="columns",inplace=True)
del dfsvm["a"]

In [102]:
words = dfsvm["Tweet"]
scores = dfsvm["Scores"]

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split

In [104]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=3.0)
model = GridSearchCV(SVC(), {'gamma':[1, 0.1, 0.001, 0.0001, 0.00001],
                                         'kernel':['linear', 'rbf']},
                                 refit=True, verbose=0)

In [106]:
words_f = vectorizer.fit_transform(raw_documents=words)

In [107]:
x_train, x_test, y_train, y_test = train_test_split(words_f, scores,
                                                            test_size=0.2, shuffle=True)

In [108]:
model.fit(x_train, y_train)

c:\users\tuan tamvan\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'gamma': [1, 0.1, 0.001, 0.0001, 1e-05],
                         'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [109]:
model = model.best_estimator_

In [110]:
model.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [111]:
y_pred = model.predict(x_test)

In [112]:
report = classification_report(y_test, y_pred, output_dict=True)

In [113]:
report

{'-1.0': {'precision': 0.7648809523809523,
  'recall': 0.8862068965517241,
  'f1-score': 0.8210862619808306,
  'support': 290},
 '1.0': {'precision': 0.8216216216216217,
  'recall': 0.658008658008658,
  'f1-score': 0.730769230769231,
  'support': 231},
 'accuracy': 0.7850287907869482,
 'macro avg': {'precision': 0.793251287001287,
  'recall': 0.7721077772801911,
  'f1-score': 0.7759277463750307,
  'support': 521},
 'weighted avg': {'precision': 0.7900385235797903,
  'recall': 0.7850287907869482,
  'f1-score': 0.7810416665683939,
  'support': 521}}

In [19]:
df1

,Tweet,Scores
4,liat timeline kpu bakal tabrak hehe,-1
5,komplek mega mai mana suaranya salah input kpu...,-1
6,tps gerung utara gerung lombok barat ntb input...,-1
7,malah lebih dahsyat salah inputnya,-1
8,salah kpu input data formulir c manusiawi,-1
9,kpuid sedikit bocor,-1
10,salah kpuid kpu tanya komisi ii dpr pilih manu...,-1
11,rakyat surabaya tuntut suara dirampok inilah h...,-1
12,pemilu kali beres situngnya beres rekap kabupa...,-1
13,tolak narasi curang pmii gelar aksi dukung kpu...,-1
